In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import os
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)

1.4.0-rc0


In [2]:
from sys import path as import_path
import_path.append('/home/robin/code/models/research/slim/')
from nets import inception
# from datasets import dataset_utils
# from tensorflow.contrib.layers.python.layers import layers as layers_lib
slim = tf.contrib.slim
image_size = inception.inception_v3.default_image_size

In [3]:
batch_size = 100
import numpy as np

def gen_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames)
    input_mean = 128
    input_std = 128
    def parser(record):
        keys_to_features = {
          'height': tf.FixedLenFeature((), tf.int64),
          'width': tf.FixedLenFeature((), tf.int64),
          'image_raw': tf.FixedLenFeature((), tf.string),
          'lable': tf.FixedLenFeature((), tf.int64)
        }
        parsed = tf.parse_single_example(record, keys_to_features)
        image = parsed["image_raw"]
        image = tf.decode_raw(image, tf.uint8)
        image = tf.reshape(image, [299, 299, 3])
        image = tf.cast(image, tf.float32)
        image = tf.subtract(image, input_mean)
        image = tf.multiply(image, 1.0/input_std)
        label = tf.cast(parsed["lable"], tf.int32)
        return image, label
    
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=3000)
    dataset = dataset.batch(batch_size)
    return dataset

In [4]:
# def load_pretrained_net():
#     checkpoint_exclude_scopes=["InceptionV3/Logits", "InceptionV3/AuxLogits"]
#     exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

#     variables_to_restore = []
#     for var in slim.get_model_variables():
#         excluded = False
#         for exclusion in exclusions:
#             if var.op.name.startswith(exclusion):
#                 excluded = True
#                 break
#         if not excluded:
#             variables_to_restore.append(var)
#     return slim.assign_from_checkpoint_fn(model_path, variables_to_restore)

def get_variables_to_restore():
    checkpoint_exclude_scopes=["InceptionV3/Logits", "InceptionV3/AuxLogits"]
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    return variables_to_restore

In [5]:
def get_fc_output(images, num_classes, dropout_keep_prob=1.0, is_training=True, reuse=False):
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, endpoints = inception.inception_v3(
            images,
            dropout_keep_prob=dropout_keep_prob,
            num_classes=num_classes,
            is_training=is_training,
            reuse=reuse
        )
    net = endpoints['Mixed_7c'] # take the layer just before logits
    net = tf.stop_gradient(net) # we keep inception model fixed
    return logits

In [6]:
logs_path = '/home/robin/tmp/train_logs'
model_path = '/home/robin/project/inception_v3.ckpt'

num_classes = 5
tf.logging.set_verbosity(tf.logging.DEBUG)

_data = tf.placeholder(tf.float32, [None, image_size, image_size, 3])
_labels = tf.placeholder(tf.int32, [None])
_keep_prob = tf.placeholder(tf.float32)

In [7]:
# logits_train = get_fc_output(_data, num_classes, dropout_keep_prob=_keep_prob, is_training=True, reuse=False)
# logits_test = get_fc_output(_data, num_classes, dropout_keep_prob=_keep_prob, is_training=False, reuse=True)

logits = get_fc_output(_data, num_classes, _keep_prob)
logits_train = logits
logits_test = logits

In [8]:
## Loss
one_hot_labels = slim.one_hot_encoding(_labels, num_classes)
total_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, labels=one_hot_labels))
# total_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, labels=one_hot_labels)

tf.summary.scalar('losses/Total_Loss', total_loss)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(total_loss)  
merged_summary_op = tf.summary.merge_all()

In [9]:
## validation 
score = tf.nn.softmax(logits_test)
correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(one_hot_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
## dataset 
with tf.name_scope('train_data'):
    train_dataset   = gen_dataset(['./retrain_flowers/flower_train.tfrecords'])
    train_iterator  = train_dataset.make_initializable_iterator()
    next_train_batch  = train_iterator.get_next()

with tf.name_scope('val_data'):
    val_dataset     = gen_dataset(['./retrain_flowers/flower_test.tfrecords'])
    val_iterator    = val_dataset.make_initializable_iterator()
    next_val_batch  = val_iterator.get_next()

In [11]:
## accuracy
def get_test_acc(sess, test_op):
    sess.run(val_iterator.initializer)
    total = 0.0
    i = 0
    total_num = 0
    while True:
        try:
            images, labels = sess.run(next_val_batch)
            acc = sess.run(test_op, feed_dict={_data:images, _labels:labels, _keep_prob:1.0})
            total += acc * images.shape[0]
            total_num += images.shape[0]
            i += 1
        except Exception:
            break
    acc_ = total/total_num  
    return acc_

In [12]:
final_model_path = "/home/robin/tmp/fine_tuned_model.ckpt"
model_saver = tf.train.Saver()

In [13]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

with tf.Session(config=config) as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    variables_to_restore = get_variables_to_restore()
    restorer = tf.train.Saver(variables_to_restore) 
    restorer.restore(sess, model_path)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    acc = get_test_acc(sess, accuracy)
    print('before fine tune: acc', acc)  

    sess.run(train_iterator.initializer)
    num_epoc = 0
    i = 0
    start = time.time()
    while num_epoc < 100:
        try:
            images, labels = sess.run(next_train_batch)
            _, summary = sess.run([train_op, merged_summary_op], feed_dict={_data:images, 
                                                                            _labels:labels, 
                                                                            _keep_prob:0.9})
            summary_writer.add_summary(summary, i)
            i += 1
            if i%500 == 0:
                print('Step: ', i)
        except tf.errors.OutOfRangeError:
            acc = get_test_acc(sess, accuracy)
            num_epoc += 1
            print('epic %d, time used: %d, run valiation acc: %f'%(num_epoc, time.time() - start, acc))   
            start = time.time()

            sess.run(train_iterator.initializer)
            images, labels = sess.run(next_train_batch)
            sess.run([train_op, merged_summary_op], feed_dict={_data:images, 
                                                               _labels:labels, 
                                                               _keep_prob:0.9})

    acc = get_test_acc(sess, accuracy)
    print('after fine tune: acc', acc) 
    save_path = model_saver.save(sess, final_model_path)

INFO:tensorflow:Restoring parameters from /home/robin/project/inception_v3.ckpt
before fine tune: acc 0.210597824791
epic 1, time used: 40, run valiation acc: 0.245924
epic 2, time used: 36, run valiation acc: 0.251359
epic 3, time used: 36, run valiation acc: 0.256793
epic 4, time used: 36, run valiation acc: 0.251359
epic 5, time used: 36, run valiation acc: 0.267663
epic 6, time used: 36, run valiation acc: 0.262228
epic 7, time used: 36, run valiation acc: 0.298913
epic 8, time used: 36, run valiation acc: 0.324728
epic 9, time used: 36, run valiation acc: 0.375000
epic 10, time used: 36, run valiation acc: 0.387228
epic 11, time used: 36, run valiation acc: 0.403533
epic 12, time used: 36, run valiation acc: 0.442935
epic 13, time used: 36, run valiation acc: 0.415761
epic 14, time used: 36, run valiation acc: 0.563859
epic 15, time used: 36, run valiation acc: 0.589674
epic 16, time used: 36, run valiation acc: 0.611413
epic 17, time used: 36, run valiation acc: 0.610054
Step:  5

In [15]:
with tf.Session(config=config) as sess:
    # Initialize variables
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

    # Restore model weights from previously saved model
    load_path = model_saver.restore(sess, final_model_path)
    acc = get_test_acc(sess, accuracy)
    print('after fine tune: acc', acc)

INFO:tensorflow:Restoring parameters from /home/robin/tmp/fine_tuned_model.ckpt
after fine tune: acc 0.796195645695
